# Pre-Training BERT with Hugging Face Transformers and Habana Gaudi

In this tutorial, you will learn how to pre-train [BERT-base](https://huggingface.co/bert-base-uncased) from scratch using a Habana Gaudi-based [DL1 instance](https://aws.amazon.com/ec2/instance-types/dl1/) on AWS to take advantage of the cost performance benefits of Gaudi. We will use the Hugging Faces [Transformers](https://huggingface.co/docs/transformers), [Optimum Habana](https://huggingface.co/docs/optimum/main/en/habana_index) and [Datasets](https://huggingface.co/docs/datasets) library to pre-train a BERT-base model using masked-language modelling one of the two original BERT pre-training tasks. Before we get started, we need to set up the deep learning environment.

You will learn how to:

1. [Prepare the dataset](#1-prepare-the-dataset)
2. [Train a Tokenizer](#2-train-a-tokenizer)
3. [Preprocess the dataset](#3-preprocess-the-dataset)
4. [Pre-train BERT on Habana Gaudi](#4-pre-train-bert-on-habana-gaudi)

_Note: Step 1 to 3 can/should be run on a different instance size those are CPU intensive tasks._

![pre-training overview](../assets/pre-training.png)

**Requirements**

Before we can start make sure you have met the following requirements

* AWS Account with quota for [DL1 instance type](https://aws.amazon.com/ec2/instance-types/dl1/)
* [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) installed
* AWS IAM user [configured in CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) with permission to create and manage ec2 instances

**Helpful Resources**

* [Setup Deep Learning environment for Hugging Face Transformers with Habana Gaudi on AWS](https://www.philschmid.de/getting-started-habana-gaudi)
Gaudi on AWS](https://www.philschmid.de/getting-started-habana-gaudi)
* [Deep Learning setup made easy with EC2 Remote Runner and Habana Gaudi](https://www.philschmid.de/habana-gaudi-ec2-runner)
* [Optimum Habana Documentation](https://huggingface.co/docs/optimum/main/en/habana_index)
* [Pre-training script](./scripts/run_mlm.py)



## What is BERT? 

BERT, short for Bidirectional Encoder Representations from Transformers, is a Machine Learning (ML) model for natural language processing. It was developed in 2018 by researchers at Google AI Language and serves as a swiss army knife solution to 11+ of the most common language tasks, such as sentiment analysis and named entity recognition.

Read more about BERT in our [BERT 101 🤗 State Of The Art NLP Model Explained](https://huggingface.co/blog/bert-101) blog.

## What is a Masked Language Modeling (MLM)?

MLM enables/enforces bidirectional learning from text by masking (hiding) a word in a sentence and forcing BERT to bidirectionally use the words on either side of the covered word to predict the masked word.

**Masked Language Modeling Example:**

```bash
“Dang! I’m out fishing and a huge trout just [MASK] my line!”
```
Read more about Masked Language Modeling [here](https://huggingface.co/blog/bert-101).

--- 

Lets get started. 🚀

_Note: Step 1 to 3 where run on a AWS c6i.12xlarge instance._

## 1. Prepare the dataset

The Tutorial is "split" into two parts. The first part (step 1-3) is about preparing the dataset and tokenizer. The second part (step 4) is about pre-training BERT on the prepared dataset. Before we can start with the dataset preparation we need to setup our development environment. As mentioned in the introduction you don't need to prepare the dataset on the DL1 instance and could use your notebook or desktop computer. 

As first we are going to install `transformers`, `datsets` and `git-lfs` to push our Tokenizer and dataset to the [Hugging Face Hub](https://huggingface.co) for later use.

In [ ]:
!pip install transformers datasets
!sudo apt-get install git-lfs

to finish our setup lets log into the [Hugging Face Hub](https://huggingface.co/models) to push our dataset, tokenizer, model artifacts, logs and metrics during training and afterwards to the hub. 

_To be able to push our model to the Hub, you need to register on the [Hugging Face](https://huggingface.co/join)._

We will use the `notebook_login` util from the `huggingface_hub` package to log into our account. You can get your token in the settings at [Access Tokens](https://huggingface.co/settings/tokens)

In [2]:
from huggingface_hub import notebook_login

notebook_login()


Since we are now logged in lets get the `user_id`, which will be used to push the artifacts.

In [2]:
from huggingface_hub import HfApi

user_id = HfApi().whoami()["name"]

print(f"user id '{user_id}' will be used during the example")

user id 'philschmid' will be used during the example


The [original BERT](https://arxiv.org/abs/1810.04805) was pretrained on [Wikipedia](https://huggingface.co/datasets/wikipedia) and [BookCorpus](https://huggingface.co/datasets/bookcorpus) dataset. Both datasets are available on the [Hugging Face Hub](https://huggingface.co/datasets) and can be loaded with `datasets`. 

_Note: For wikipedia we will use the `20220301`, which is different to the original split._

As a first step are we loading the dataset and merging them together to create on big dataset.

In [3]:
from datasets import concatenate_datasets, load_dataset

bookcorpus = load_dataset("bookcorpus", split="train")
wiki = load_dataset("wikipedia", "20220301.en", split="train")
wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])  # only keep the 'text' column

assert bookcorpus.features.type == wiki.features.type
raw_datasets = concatenate_datasets([bookcorpus, wiki])

Reusing dataset bookcorpus (/home/ubuntu/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/44662c4a114441c35200992bea923b170e6f13f2f0beb7c14e43759cec498700)
Reusing dataset wikipedia (/home/ubuntu/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [4]:
raw_datasets

Dataset({
    features: ['text'],
    num_rows: 80462898
})

> We are not going to do some advanced dataset preparation, like de-duplication, filtering or any other pre-processing. If you are planning to apply this notebook to train your own BERT model from scratch I highly recommend to including those data preparation steps into your workflow. This will help you improve your Language Model. 

## 2. Train a Tokenizer

To be able to train our model we need to convert our text into a tokenized format. Most Transformer models are coming with a pre-trained tokenizer, but since we are pre-training our model from scratch we also need to train a Tokenizer on our data. We can train a tokenizer on our data with `transformers` and the `BertTokenizerFast` class. 

More information about training a new tokenizer can be found in our [Hugging Face Course](https://huggingface.co/course/chapter6/2?fw=pt).

In [5]:
from tqdm import tqdm
from transformers import BertTokenizerFast

# repositor id for saving the tokenizer
tokenizer_id="bert-base-uncased-2022-habana"

# create a python generator to dynamically load the data
def batch_iterator(batch_size=10000):
    for i in tqdm(range(0, len(raw_datasets), batch_size)):
        yield raw_datasets[i : i + batch_size]["text"]

# create a tokenizer from existing one to re-use special tokens
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


We can start training the tokenizer with `train_new_from_iterator()`.


In [6]:
bert_tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
bert_tokenizer.save_pretrained("tokenizer")

100%|██████████| 8047/8047 [09:13<00:00, 14.53it/s]


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

We push the tokenizer to [Hugging Face Hub](https://huggingface.co/models) for later training our model.


In [ ]:
# you need to be logged into push the tokenizer
bert_tokenizer.push_to_hub(tokenizer_id)

## 3. Preprocess the dataset

The last step before we can get started with training our model is to pre-process/tokenize our dataset. We will use our trained tokenizer to tokenize our dataset and then push it to hub to load it easily later in our training.

In [8]:
from transformers import AutoTokenizer
import multiprocessing

# load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(f"{user_id}/{tokenizer_id}")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")
num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples["text"], return_special_tokens_mask=True, truncate=True, max_length=tokenizer.model_max_length
    )
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = raw_datasets.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features


The max length for the tokenizer is: 512


  0%|          | 0/80463 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (594 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'special_tokens_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

As data processing function will we concatenate all texts from our dataset and generate chunks of `tokenizer.model_max_length` (512).

In [ ]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= tokenizer.model_max_length:
        total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=num_proc)

print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")

The last step before we can start with out training is to push our prepared dataset to the hub.

In [19]:
# push dataset to hugging face
dataset_id=f"{user_id}/processed_bert_dataset"
tokenized_datasets.push_to_hub(f"{user_id}/processed_bert_dataset")

Pushing dataset shards to the dataset hub:   0%|          | 0/90 [00:00<?, ?it/s]

/home/ubuntu/miniconda3/lib/python3.9/site-packages/huggingface_hub/hf_api.py:1948: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(



## 4. Setup Habana Gaudi instance

> The following steps are created to be executed on a Gaudi instance.

In this example are we going to use Habana Gaudi on AWS using the DL1 instance. We already have created a blog post in the past on how to [Setup Deep Learning environment for Hugging Face Transformers with Habana Gaudi on AWS](https://www.philschmid.de/getting-started-habana-gaudi). If you haven't have read this blog post, please read it first and go through the steps on how to setup the environment. 
Or if you feel comfortable you can use the `start_instance.sh` in the root of the repository to create your DL1 instance and the continue at step  [4. Use Jupyter Notebook/Lab via ssh](https://www.philschmid.de/getting-started-habana-gaudi#4-use-jupyter-notebooklab-via-ssh) in the Setup blog post.

1. run habana docker container an mount current directory
```bash
docker run -ti --runtime=habana -e HABANA_VISIBLE_DEVICES=all -e OMPI_MCA_btl_vader_single_copy_mechanism=none --cap-add=sys_nice --net=host --ipc=host -v $(pwd):/home/ubuntu/dev --workdir=/home/ubuntu/dev vault.habana.ai/gaudi-docker/1.4.1/ubuntu20.04/habanalabs/pytorch-installer-1.10.2:1.4.1-11
```
2. install juptyer
```bash
pip install jupyter
```

3. clone repository
```bash
git clone https://github.com/philschmid/deep-learning-habana-huggingface.git
cd fine-tuning
```

4. run jupyter notebook
```bash
jupyter notebook --allow-root
#         http://localhost:8888/?token=f8d00db29a6adc03023413b7f234d110fe0d24972d7ae65e
```
4. continue here

_**NOTE**: The following steps assume that the code/cells are running on a gaudi instance with access to HPUs_

As first lets make sure we have access to the HPUs.

In [1]:
import habana_frameworks.torch.core as htcore

print(f"device available:{htcore.is_available()}")
print(f"device_count:{htcore.get_device_count()}")

device available:True
device_count:8


## 5. Pre-train BERT on Habana Gaudi

Normally you would use the [Trainer](https://huggingface.co/docs/transformers/v4.19.4/en/main_classes/trainer#transformers.Trainer) and [TrainingArguments](https://huggingface.co/docs/transformers/v4.19.4/en/main_classes/trainer#transformers.TrainingArguments). Since we are using the `optimum-habana` library, we can use the [GaudiTrainer]() and [GaudiTrainingArguments]() instead. The `GaudiTrainer` is a wrapper around the [Trainer](https://huggingface.co/docs/transformers/v4.19.4/en/main_classes/trainer#transformers.Trainer that allows you to pre-traing or fine-tune a transformer model on a Habana Gaudi instances.

```diff
-from transformers import Trainer, TrainingArguments 
+from optimum.habana import GaudiTrainer, GaudiTrainingArguments

# define the training arguments
-training_args = TrainingArguments(
+training_args = GaudiTrainingArguments(
+  use_habana=True,
+  use_lazy_mode=True,
+  gaudi_config_name=path_to_gaudi_config,
  ...
)

# Initialize our Trainer
-trainer = Trainer(
+trainer = GaudiTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
    ... # other arguments
)
```

The `DL1` instance we use has 8 available HPU-cores meaning we can leverage distributed data-parallel training for our model. 
To run our training as distributed training we need to create a training script, which can be used with multiprocessing to run on all HPUs. 
We have created a [scripts/run_mlm.py](https://github.com/philschmid/deep-learning-habana-huggingface/blob/master/pre-training/scripts/run_mlm.py) implementing masked-language modeling using the `GaudiTrainer`. To executed our distributed training we use the `DistributedRunner` runner from `optimum-habana` and pass our arguments. Alternatively you could check-out the [gaudi_spawn.py](https://github.com/huggingface/optimum-habana/blob/main/examples/gaudi_spawn.py) in the [optimum-habana](https://github.com/huggingface/optimum-habana) repository.


In [ ]:
from huggingface_hub import HfFolder
from optimum.habana.distributed import DistributedRunner


max_steps=10_000
world_size=8 # Number of HPUs to use (1 or 8)
per_device_train_batch_size=16
learning_rate=3e-5
hf_hub_token=HfFolder.get_token()

# define distributed runner
distributed_runner = DistributedRunner(
    command_list=[f"scripts/run_mlm.py --dataset_id {dataset_id} --repository_id {tokenizer_id} --per_device_train_batch_size {per_device_train_batch_size} --learning_rate {learning_rate} --hf_hub_token {hf_hub_token}"],
    world_size=world_size,
    use_mpi=True,
    multi_hls=False,
)

# start job
ret_code = distributed_runner.run()